In [1]:
from my_methods import *
import requests
import io

#### In this notebook I search for genes strongly connected in the add measure, ignored by the mul measure

In [2]:
df = pd.read_table('data/SGA_ExE.txt')
df = df.dropna()            # Elimino filas con NaN
df.reset_index(inplace=True, drop=True)
##################################
df['Mul'] = df['Double mutant fitness']-df['Query single mutant fitness (SMF)']*df['Array SMF']         # columna con interacciones mul
df['Add'] = df['Double mutant fitness']-(df['Query single mutant fitness (SMF)']+df['Array SMF']-1)     # columna con interacciones add
##################################
std_mul = df['Mul'].std()           # Desviaciones estandar de cada columna
std_add = df['Add'].std()
##################################
df['Query allele name'] = df['Query allele name'].map(lambda x: x.split('-')[0])        # Cambio de nombre
df['Array allele name'] = df['Array allele name'].map(lambda x: x.split('-')[0])
##################################
df = df.loc[df['P-value']<0.05]             # Filtro P-value
df.head()

,Query Strain ID,Query allele name,Array Strain ID,Array allele name,Arraytype/Temp,Genetic interaction score (ε),P-value,Query single mutant fitness (SMF),Array SMF,Double mutant fitness,Double mutant fitness standard deviation,Mul,Add
0,YAL001C_tsq508,tfc3,YBL023C_tsa111,mcm2,TSA30,-0.0348,5.042000e-03,0.8285,0.9254,0.7319,0.0102,-0.034794,-0.0220
1,YAL001C_tsq508,tfc3,YBL026W_tsa1065,lsm2,TSA30,-0.3529,3.591000e-06,0.8285,0.9408,0.4266,0.0790,-0.352853,-0.3427
6,YAL001C_tsq508,tfc3,YBL034C_tsa950,stu1,TSA30,-0.1294,1.931000e-02,0.8285,0.6690,0.4249,0.0482,-0.129366,-0.0726
12,YAL001C_tsq508,tfc3,YBL076C_tsa275,ils1,TSA30,-0.0250,1.301000e-04,0.8285,0.8097,0.6458,0.0054,-0.025036,0.0076
15,YAL001C_tsq508,tfc3,YBL097W_tsa510,brn1,TSA30,-0.0808,5.582000e-15,0.8285,0.5464,0.3719,0.0077,-0.080792,-0.0030


In [3]:
def df_to_dictionary(csv, model, pvalue=1):
    df = csv.loc[csv[ 'P-value' ]<pvalue]
    df.reset_index(inplace=True, drop=True)
    query_dic = {}
    array_dic = {}
    for i in range(len(df)):
        if model == 'mul':
            interaction = df.loc[i, 'Double mutant fitness']-df.loc[i, 'Query single mutant fitness (SMF)']*df.loc[i, 'Array SMF']
        if model == 'add':
            interaction = df.loc[i, 'Double mutant fitness']-(df.loc[i, 'Query single mutant fitness (SMF)']+df.loc[i, 'Array SMF']-1)
        if not (np.isnan(interaction) or np.isinf(interaction)):
            query = df.loc[i, 'Query allele name']
            array = df.loc[i, 'Array allele name']
            ########################################################
            if query not in query_dic.keys():
                query_dic[query]={}
            if array not in query_dic[query].keys():
                query_dic[query][array]=interaction
            elif abs(interaction) > abs(query_dic[query][array]):
                query_dic[query][array]=interaction
            ########################################################
            if array not in array_dic.keys():
                array_dic[array]={}
            if query not in array_dic[array].keys():
                array_dic[array][query]=interaction
            elif abs(interaction) > abs(array_dic[array][query]):
                array_dic[array][query]=interaction
            ########################################################
    return query_dic, array_dic

In [4]:
qE, aE = df_to_dictionary(df, model='mul', pvalue=0.05)
ExE_mul = join_pcc(qE, aE)

In [5]:
qE, aE = df_to_dictionary(df, model='add', pvalue=0.05)
ExE_add = join_pcc(qE, aE)

In [ ]:
# v_m = []
# v_a = []
# for k in ExE_mul.keys():
#     v_m.extend(ExE_mul[k].values())
# for k in ExE_add.keys():
#     v_a.extend(ExE_add[k].values())
# v_m = np.array(v_m)
# v_a = np.array(v_a)
# np.mean(v_m), np.mean(v_a)

In [6]:
# ExE_mul, mu_mul, sigma_mul = normalize_dictionary(ExE_mul)
# ExE_add, mu_add, sigma_add = normalize_dictionary(ExE_add)

In [9]:
2*std_mul

0.16560200705000264

In [10]:
for gene in ExE_mul.keys():
    ExE_mul[gene] = dict(filter(lambda x: abs(x[1])>2*std_mul, ExE_mul[gene].items()))

for gene in ExE_add.keys():
    ExE_add[gene] = dict(filter(lambda x: abs(x[1])>2*std_add, ExE_add[gene].items()))

In [8]:
# potential_hubs0 = []
# for gene in ExE_add.keys():
#     try:
# ####### Busco los que en add tienen 10 veces mas interactores que en mul
#         if len(ExE_mul[gene]) > 0 and len(ExE_mul[gene]) < 0.1 * len(ExE_add[gene]):
#             potential_hubs0.append(gene)
# ####### Busco los que en mul tienen 0 interactores y mas de 30
#         if len(ExE_mul[gene]) == 0 and len(ExE_add[gene])>30:
#             potential_hubs0.append(gene)
#     except:
#         pass
# potential_hubs0 = list(set(potential_hubs0))
# len(potential_hubs0)

14

In [11]:
potential_hubs0 = []
for gene in ExE_add.keys():
    try:
####### De aquellos genes con mas de 30 interactores en add se buscan los genes que en add tienen mas de 10 veces la cantidad de interactores predicha por mul
####### Busco los que en add tienen 10 veces mas interactores que en mul
        if len(ExE_add[gene])>30 and len(ExE_mul[gene]) > 0 and len(ExE_mul[gene]) < 0.1 * len(ExE_add[gene]):
            potential_hubs0.append(gene)
####### Busco los que en mul tienen 0 interactores y mas de 30
        if len(ExE_mul[gene]) == 0 and len(ExE_add[gene])>30:
            potential_hubs0.append(gene)
    except:
        pass
potential_hubs0 = list(set(potential_hubs0))
len(potential_hubs0)

13

In [12]:
table = pd.DataFrame(
    {'Candidate Hubs':[],
    '# of interactors (eM)':[],
    '# of interactors (eA)':[],
    '# of common interactors':[],
    }
)
for i in range(len(potential_hubs0)):
    gene = potential_hubs0[i]
    # int_mul = ExE_mul[gene].keys()
    # int_add = ExE_add[gene].keys()
    int_mul = len(ExE_mul[gene])
    int_add = len(ExE_add[gene])
    intersection = set(ExE_mul[gene].keys()).intersection(set(ExE_add[gene].keys()))
    # table.loc[i,] = [i, len(int_mul), len(int_add), len(intersection)]
    table.loc[i,] = [i, int_mul, int_add, len(intersection)]
table['Candidate Hubs'] = list(map(lambda x: x.split('-')[0], potential_hubs0))
table[table.columns[1:4]] = table[table.columns[1:4]].astype('int32')
table

,Candidate Hubs,# of interactors (eM),# of interactors (eA),# of common interactors
0,rok1,0,71,0
1,mpp10,0,86,0
2,noc4,0,73,0
3,tim17,1,53,0
4,erg12,5,75,0
5,rcl1,2,56,1
6,rrp7,0,49,0
7,tif35,0,78,0
8,pop6,1,33,1
9,vrg4,2,58,2


In [10]:
table.to_csv('potential_hubs.csv')

In [14]:
## Hago el analisis inverso, para hallar los posibles hubs de mul
potential_hubs_mul = []
for gene in ExE_mul.keys():
    try:
####### De aquellos genes con mas de 30 interactores en add se buscan los genes que en add tienen mas de 10 veces la cantidad de interactores predicha por mul
####### Busco los que en add tienen 10 veces mas interactores que en mul
        if len(ExE_mul[gene])>30 and len(ExE_add[gene]) > 0 and len(ExE_add[gene]) < 0.1 * len(ExE_mul[gene]):
            potential_hubs_mul.append(gene)
####### Busco los que en mul tienen 0 interactores y mas de 30
        if len(ExE_add[gene]) == 0 and len(ExE_mul[gene])>30:
            potential_hubs_mul.append(gene)
    except:
        pass
potential_hubs_mul = list(set(potential_hubs_mul))
len(potential_hubs_mul) # No hay hubs potenciales en mul

0

In [15]:
menos = ['yef3', 'plp2', 'kae1', 'yef3']

0.08280100352500132

In [16]:
yef_df = df.loc[((df['Query allele name'].str.contains('yef3')) | (df['Array allele name'].str.contains('yef3'))) & (abs(df['Mul'])>2*std_mul)]
yef_df

,Query Strain ID,Query allele name,Array Strain ID,Array allele name,Arraytype/Temp,Genetic interaction score (ε),P-value,Query single mutant fitness (SMF),Array SMF,Double mutant fitness,Double mutant fitness standard deviation,Mul,Add
484263,YLR249W_tsq584,yef3,YHR107C_tsa50,cdc12,TSA26,-0.2446,5.172000e-26,0.5436,0.8644,0.2253,0.0212,-0.244588,-0.1827
484451,YLR249W_tsq584,yef3,YNL118C_tsa754,dcp2,TSA26,-0.1779,1.380000e-04,0.5436,0.5762,0.1353,0.1140,-0.177922,0.0155


In [11]:
# list_save(list(map(lambda x: x.split('-')[0].upper(), potential_hubs0)), 'potential_hubs.txt')
list_save(list(map(lambda x: x.upper(), potential_hubs0)), 'potential_hubs.txt')

In [12]:
!find_enrichment.py potential_hubs.txt ~/MyStuff/tesis/Jupyter/GO/population.txt ~/MyStuff/tesis/Jupyter/GO/pairs.txt --obo ~/MyStuff/tesis/Jupyter/GO/go.obo > potential_result.txt
!tail -n +90 potential_result.txt | head -n 5 > potential_result.txt_parsed.txt

In [13]:
potential_hubs_enrichment = pd.read_table('potential_result.txt_parsed.txt')
potential_hubs_enrichment

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,p_fdr_bh,study_items
0,GO:0022613,BP,e,ribonucleoprotein complex biogenesis,8/13,224/6982,1.110000e-09,4,8,0.000006,0.000006,0.000006,0.000004,"BMS1, IMP4, MPP10, NOC4, RCL1, ROK1, RRP7, TRM112"
1,GO:0006364,BP,e,rRNA processing,9/13,363/6982,1.500000e-09,9,9,0.000008,0.000007,0.000008,0.000004,"BMS1, IMP4, MPP10, NOC4, POP6, RCL1, ROK1, RRP..."
2,GO:0044085,BP,e,cellular component biogenesis,8/13,250/6982,2.670000e-09,3,8,0.000014,0.000013,0.000014,0.000004,"BMS1, IMP4, MPP10, NOC4, RCL1, ROK1, RRP7, TRM112"
3,GO:0000480,BP,e,endonucleolytic cleavage in 5'-ETS of tricistr...,5/13,35/6982,2.940000e-09,11,5,0.000015,0.000015,0.000015,0.000004,"BMS1, MPP10, NOC4, RCL1, ROK1"


In [17]:
potential_hubs_enrichment[['GO', 'NS', 'name', 'ratio_in_study', 'p_bonferroni', 'study_items']].to_csv('enrichment.csv')

In [18]:
# jorge = {'mpp10', 'trm112', 'erg12', 'tif35', 'noc4', 'rok1', 'bms1', 'vrg4', 'rcl1', 'tim17', 'imp4', 'rrp7', 'pop6'}
# potential_hubs = set(map(lambda x: x.split('-')[0], potential_hubs0))

In [1]:
# print('Comunes:\n', jorge.intersection(potential_hubs))
# # tif no esta en procesoma, pero interactua con el ribosoma
# # tim17 es de la mitocondria, nada que ver

In [2]:
# print('Mios, no de Jorge:\n', potential_hubs.difference(jorge))
# # ded1 en procesamiento de pre-mRNA
# # mcm6 en inicio de la replicacion
# # tfb1 en reparacion de nucleotidos y subunidad de un factor de transcripcion
# # yef3 es factor de elongacion

In [3]:
# print('Jorge, no mios:\n', jorge.difference(potential_hubs))
# # bms1 esta en sintesis de subunidades ribosomales y procesamiento de rRNA
# # erg12 Mevalonate kinase; acts in the biosynthesis of isoprenoids and sterols
# # imp4 Component of the SSU processome
# # noc4 Nucleolar protein; forms a complex with Nop14p that mediates maturation and nuclear export of 40S ribosomal subunits

In [ ]:
### Comprobando la existencia de estas interacciones para add a traves de la api de STRING

In [15]:
# percentage_predicted = {}
# exceptions = []
# for hub in potential_hubs0:
#     try:
#         gene = hub.split('-')[0].upper()
#         response = requests.get(f'https://string-db.org/api/tsv/interaction_partners?identifiers={gene}&species=4932&limit=100')
#         table = pd.read_table(io.StringIO(response.content.decode('utf-8')))
#         df = table.loc[(table['escore']>0.5) | (table['dscore']>0.5) | (table['tscore']>0.5)]
#         interactors_predicted = set(ExE_add[hub].keys())      ########## Recuerda que aqui estas usando ExE
#         interactors_predicted = set(map(lambda x: x.split('-')[0].upper(), interactors_predicted))
#         interactors_db = set(df['preferredName_B'])
#         common = interactors_db.intersection(interactors_predicted)
#         percentage_predicted[hub] = len(common)/len(interactors_predicted)
#     except:
#         exceptions.append(hub)